# Filter rooftop data to Agra and Solan

## 1. Import packages and set paths

In [ ]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
from tqdm import tqdm

from rooftop_tools.utils_rooftop import (
    get_matched_rooftop_centroids_from_s2_file,
    get_overlapping_s2_cell_ids,
)

In [ ]:
# set paths
FH_SAMPLING_FOLDER = Path("../") / "data" / "fortify_data"
SHRUG_district_path = (
    FH_SAMPLING_FOLDER / "Shape files/shrug-pc11dist-poly-shp/district.shp"
)
SHRUG_sub_path = (
    FH_SAMPLING_FOLDER / "Shape files/shrug-pc11subdist-poly-shp/subdistrict.shp"
)

## 2. Import and clean SHRUG data

In [ ]:
# import SHRUG district boundary and select just the pilot districts
SHRUG_districts = gpd.read_file(SHRUG_district_path)
agra = SHRUG_districts[SHRUG_districts["d_name"] == "Agra"]
# print the gdf to make sure there were no spelling mistakes in district names
agra

In [ ]:
SHRUG_subs = gpd.read_file(SHRUG_sub_path)
pilot_subs = SHRUG_subs[
    (SHRUG_subs["pc11_d_id"] == "031") & (SHRUG_subs["sd_name"] == "Solan")
]
pilot_subs["d_name"] = "Solan"
pilot_subs

In [ ]:
pilot_areas = pd.concat([agra, pilot_subs])
pilot_areas

## 3. For each s2 file, filter for rooftops in districts

In [ ]:
s2_cell_ids = get_overlapping_s2_cell_ids(pilot_areas)
matched_rooftop_centroids_gdf_list = []

for s2_cell_id in tqdm(s2_cell_ids):
    matched_rooftop_centroids_gdf = get_matched_rooftop_centroids_from_s2_file(
        s2_file_dir=FH_SAMPLING_FOLDER / "Rooftop Data",
        s2_cell_id=s2_cell_id,
        boundaries_gdf=pilot_areas,
    )
    matched_rooftop_centroids_gdf_list.append(matched_rooftop_centroids_gdf)

In [ ]:
for gdf in matched_rooftop_centroids_gdf_list:
    gdf.to_crs(epsg=4326, inplace=True)

In [ ]:
# concatenate the gdfs
matched_rooftop_centroids_gdf = pd.concat(
    matched_rooftop_centroids_gdf_list, ignore_index=True
)
# Save the matched rooftops data
matched_rooftop_centroids_gdf.to_parquet(
    FH_SAMPLING_FOLDER / "Cleaned rooftop data" / "Rooftops in Agra and Solan.parquet"
)